In [2]:
%load_ext autoreload
%autoreload 2

import os, sys
# sys.path.append(os.getcwd())

for line in sys.path:
     print(line)

/home/student/cs4065/lib/python35.zip
/home/student/cs4065/lib/python3.5
/home/student/cs4065/lib/python3.5/plat-x86_64-linux-gnu
/home/student/cs4065/lib/python3.5/lib-dynload
/usr/lib/python3.5
/usr/lib/python3.5/plat-x86_64-linux-gnu

/home/student/cs4065/lib/python3.5/site-packages
/home/student/cs4065/lib/python3.5/site-packages/IPython/extensions
/home/student/.ipython
/home/student/pylib/cs4065


In [3]:
import pandas as pd
import numpy as np

from src.VideoReader import VideoReader
from src.Video import Video
from src.histograms import compute_histograms
from src import preprocess as prep
import cv2
import os

SyntaxError: invalid syntax (Video.py, line 5)

In [122]:
def find(target, videos):
    video_path = os.path.join(target + ".mp4")
    target_video = VideoReader()
    target_video.open(video_path)
    frames = target_video.get_frames()
#     todo: average per novelty section in video?
    first_frame = next(frames)
    
    target_histograms = compute_histograms(first_frame)
    
    distances = []
    for video in videos:
        segment_dist = []
        for segment in video.segments:
#             zoiets? snap de tuple niet helemaal
#             hist = segment.get_hist((1,0))
            hist = target_histograms[0]
    
            dist = 0
            for h in range(len(hist)):
#                   Method kiezen
                dist += cv2.compareHist(target_histograms[0][h], hist, cv2.HISTCMP_INTERSECT)
            segment_dist.append(dist)
        
        distance.append(segment_dist)
    
    for video in videos:
        for segment in video.segments:
            for i_sub_hist in range(1,5):
                nr_hists = len(target_histograms[0])
                
                dist = 0
                for h in range(nr_hists):
#                     tuple?
                    segment_sub_hist = segment.get_hist((0,0))
#                   Method kiezen
                    dist += cv2.compareHist(target_histograms[i_sub_hist][h], segment_sub_hist, cv2.HISTCMP_INTERSECT)
            
        

# vid = prep.process_video("00001")

find(r'data/movies/00001', [])